In [71]:
# CONFIG FILE IMPORT
import sys
import os
root_dir = os.path.dirname(os.getcwd()) # Get to the root directory
config_path = os.path.join(root_dir, 'config') # Get inside the config folder
sys.path.append(config_path) 
import userdata_config as cfg # Import everything from the config file

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Graphic settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Libraries imported successfully")
df = pd.read_csv('data/turin_solar_data_20260213_160518.csv')

print(f"Loaded Dataset {df.shape[0]} rows, {df.shape[1]} columns")

Libraries imported successfully
Loaded Dataset 28 rows, 15 columns


In [59]:
df.head(5)

,city,temperature,humidity,wind_speed,cloudcover,uv_index,observation_time,date,hour,month,solar_angle,cloud_factor,temp_efficiency,uv_factor,solar_potential
0,Turin,2.0,70,3,80,0.1,2026-02-01 12:00:00,2026-02-01,12,2,0.42,0.2,1.02,0.1,0.08568
1,Turin,3.0,68,4,10,1.2,2026-02-02 12:00:00,2026-02-02,12,2,0.44,1.0,1.02,1.2,0.44880
2,Turin,4.0,65,5,0,1.6,2026-02-03 12:00:00,2026-02-03,12,2,0.46,1.0,1.01,1.6,0.46460
3,Turin,3.5,72,6,100,0.0,2026-02-04 12:00:00,2026-02-04,12,2,0.48,0.0,1.02,0.0,0.00000
4,Turin,5.0,60,4,20,1.4,2026-02-05 12:00:00,2026-02-05,12,2,0.50,0.9,1.01,1.4,0.45450


In [60]:
print(df.columns.tolist())

['city', 'temperature', 'humidity', 'wind_speed', 'cloudcover', 'uv_index', 'observation_time', 'date', 'hour', 'month', 'solar_angle', 'cloud_factor', 'temp_efficiency', 'uv_factor', 'solar_potential']


In [61]:
df.dtypes

city                 object
temperature         float64
humidity              int64
wind_speed            int64
cloudcover            int64
uv_index            float64
observation_time     object
date                 object
hour                  int64
month                 int64
solar_angle         float64
cloud_factor        float64
temp_efficiency     float64
uv_factor           float64
solar_potential     float64
dtype: object

In [62]:
# ============================================
# Q1: Preliminary Meteorological Analysis
# ============================================

print("=" * 50)
print("WEATHER STATISTICS TURIN - FEBRUARY 2026")
print("=" * 50)

WEATHER STATISTICS TURIN - FEBRUARY 2026


In [63]:
print("=" * 60)
print("TEMPERATURE STATISTICS")
print("=" * 60)

print(f"\nTEMPERATURE:")
print(f"   - Average: {df['temperature'].mean():.1f}°C")
print(f"   - Maximum: {df['temperature'].max():.1f}°C")
print(f"   - Minimum: {df['temperature'].min():.1f}°C")

# Temperature distribution by percentiles
print(f"\nTEMPERATURE PERCENTILES:")
percentiles = [0, 10, 25, 50, 75, 90, 100]
for p in percentiles:
    print(f"   {p:3d}th percentile: {df['temperature'].quantile(p/100):.1f}°C")

TEMPERATURE STATISTICS

TEMPERATURE:
   - Average: 9.0°C
   - Maximum: 14.0°C
   - Minimum: 2.0°C

TEMPERATURE PERCENTILES:
     0th percentile: 2.0°C
    10th percentile: 3.9°C
    25th percentile: 6.4°C
    50th percentile: 9.2°C
    75th percentile: 12.0°C
    90th percentile: 13.0°C
   100th percentile: 14.0°C


In [64]:
print("=" * 60)
print("CLOUD COVER STATISTICS")
print("=" * 60)

print(f"\nCLOUD COVER:")
print(f"   - Average: {df['cloudcover'].mean():.1f}")
print(f"   - Minimum: {df['cloudcover'].min():.1f}")
print(f"   - Maximum: {df['cloudcover'].max():.1f}")
print(f"   - Std Deviation: {df['cloudcover'].std():.1f}")
print(f"   - Median: {df['cloudcover'].median():.1f}")
print(f"   - Clear sky hours (0-20%): {len(df[df['cloudcover'] <= 20])} hours")
print(f"   - Cloudy hours (>60%): {len(df[df['cloudcover'] > 60])} hours")

total = len(df)
print("\nCLOUD COVER DISTRIBUTION:")
for category_name, category_range in cfg.CLOUD_CATEGORIES.items():
    category_count = len(df[(df['cloudcover'] >= category_range['min']) & 
                            (df['cloudcover'] <= category_range['max'])])
    print(f"   {category_name.replace('_', ' ').title()} ({category_range['min']}-{category_range['max']}%): "
          f"{category_count} hours ({category_count/total*100:5.1f}%)")

# Daily cloud cover patterns
daily_cloud = df.groupby('date')['cloudcover'].mean()
print(f"\nDAILY CLOUD COVER:")
print(f"   Average daily cloud cover: {daily_cloud.mean():.1f}%")
print(f"   Most cloudy day: {daily_cloud.idxmax()} ({daily_cloud.max():.1f}%)")
print(f"   Least cloudy day: {daily_cloud.idxmin()} ({daily_cloud.min():.1f}%)")

CLOUD COVER STATISTICS

CLOUD COVER:
   - Average: 34.3
   - Minimum: 0.0
   - Maximum: 100.0
   - Std Deviation: 33.4
   - Median: 22.5
   - Clear sky hours (0-20%): 14 hours
   - Cloudy hours (>60%): 6 hours

CLOUD COVER DISTRIBUTION:
   Clear Sky (0-20%): 14 hours ( 50.0%)
   Partly Cloudy (21-60%): 8 hours ( 28.6%)
   Cloudy (61-100%): 6 hours ( 21.4%)

DAILY CLOUD COVER:
   Average daily cloud cover: 34.3%
   Most cloudy day: 2026-02-04 (100.0%)
   Least cloudy day: 2026-02-03 (0.0%)


In [65]:
print("=" * 60)
print("UV INDEX STATISTICS")
print("=" * 60)

print(f"\nUV INDEX:")
print(f"   - Average: {df['uv_index'].mean():.3f}")
print(f"   - Minimum: {df['uv_index'].min()}")
print(f"   - Maximum: {df['uv_index'].max()}")
print(f"   - Std Deviation: {df['uv_index'].std():.3f}")
print(f"   - Median: {df['uv_index'].median()}")

# Hours with UV > 0
uv_positive = len(df[df['uv_index'] > 0])
total = len(df)

print(f"\nUV Activity:")
print(f"   Hours with UV > 0: {uv_positive} ({uv_positive/total*100:.2f}% of time)")
print(f"   Hours with UV = 0: {total - uv_positive} ({(total-uv_positive)/total*100:.2f}% of time)")


print(f"\nUV INTENSITY DISTRIBUTION:")
for category_name, category_range in cfg.UV_CATEGORIES.items():
    category_count = len(df[(df['uv_index'] >= category_range['min']) & 
                            (df['uv_index'] <= category_range['max'])])
    print(f"   {category_name.replace('_', ' ').title()} ({category_range['min']}-{category_range['max']}): "
          f"{category_count} hours ({category_count/total*100:5.1f}%)")


# Daily UV patterns
daily_uv = df.groupby('date')['uv_index'].mean()
print(f"\nDAILY UV PATTERNS:")
print(f"   Average daily UV: {daily_uv.mean():.2f}")
print(f"   Highest UV day: {daily_uv.idxmax()} ({daily_uv.max():.2f})")
print(f"   Lowest UV day: {daily_uv.idxmin()} ({daily_uv.min():.2f})")

UV INDEX STATISTICS

UV INDEX:
   - Average: 1.457
   - Minimum: 0.0
   - Maximum: 2.9
   - Std Deviation: 0.970
   - Median: 1.45

UV Activity:
   Hours with UV > 0: 26 (92.86% of time)
   Hours with UV = 0: 2 (7.14% of time)

UV INTENSITY DISTRIBUTION:
   Low (0-2): 19 hours ( 67.9%)
   Moderate (2-5): 11 hours ( 39.3%)
   High (5-15): 0 hours (  0.0%)

DAILY UV PATTERNS:
   Average daily UV: 1.46
   Highest UV day: 2026-02-25 (2.90)
   Lowest UV day: 2026-02-04 (0.00)


In [66]:
print("=" * 60)
print("WIND SPEED STATISTICS")
print("=" * 60)

print(f"\nWIND:")
print(f"   - Average: {df['wind_speed'].mean():.1f} km/h")
print(f"   - Minimum gusts: {df['wind_speed'].min():.1f} km/h")
print(f"   - Maximum gusts: {df['wind_speed'].max():.1f} km/h")
print(f"   - Std Deviation: {df['wind_speed'].std():.1f} km/h")
print(f"   - Median: {df['wind_speed'].median():.1f} km/h")

print(f"\nWIND INTENSITY DISTRIBUTION:")
for category_name, category_range in cfg.WIND_CATEGORIES.items():
    category_count = len(df[(df['uv_index'] >= category_range['min']) & 
                            (df['uv_index'] <= category_range['max'])])
    print(f"   {category_name.replace('_', ' ').title()} ({category_range['min']}-{category_range['max']}): "
          f"{category_count} hours ({category_count/total*100:5.1f}%)")


print(f"\nWind Conditions:")
print(f"   Calm (<5 km/h):         {calm:4d} hours ({calm/total*100:5.1f}%)")
print(f"   Light (5-20 km/h):      {light:4d} hours ({light/total*100:5.1f}%)")
print(f"   Moderate (20-40 km/h):  {moderate:4d} hours ({moderate/total*100:5.1f}%)")
print(f"   Strong (>40 km/h):      {strong:4d} hours ({strong/total*100:5.1f}%)")

WIND SPEED STATISTICS

WIND:
   - Average: 4.6 km/h
   - Minimum gusts: 3.0 km/h
   - Maximum gusts: 6.0 km/h
   - Std Deviation: 0.9 km/h
   - Median: 5.0 km/h

WIND INTENSITY DISTRIBUTION:
   Calm (0-5): 28 hours (100.0%)
   Light (5-20): 0 hours (  0.0%)
   Moderate (20-40): 0 hours (  0.0%)
   Strong (40-200): 0 hours (  0.0%)

Wind Conditions:
   Calm (<5 km/h):           12 hours ( 42.9%)
   Light (5-20 km/h):        16 hours ( 57.1%)
   Moderate (20-40 km/h):     0 hours (  0.0%)
   Strong (>40 km/h):         0 hours (  0.0%)


In [67]:
print("=" * 60)
print("SOLAR ANGLE STATISTICS")
print("=" * 60)

print(f"\nSOLAR ANGLE:")
print(f"   - Average: {df['solar_angle'].mean():.2f}")
print(f"   - Minimum: {df['solar_angle'].min():.2f}")
print(f"   - Maximum: {df['solar_angle'].max():.2f}")
print(f"   - Std Deviation: {df['solar_angle'].std():.2f}")
print(f"   - Median: {df['solar_angle'].median():.2f}")

# Solar angle by hour (average)
hourly_angle = df.groupby('hour')['solar_angle'].mean()
print(f"\n Peak solar angle hour: {hourly_angle.idxmax()}:00 ({hourly_angle.max():.3f})")
print(f" Lowest solar angle hour: {hourly_angle.idxmin()}:00 ({hourly_angle.min():.3f})")

SOLAR ANGLE STATISTICS

SOLAR ANGLE:
   - Average: 0.65
   - Minimum: 0.42
   - Maximum: 0.81
   - Std Deviation: 0.12
   - Median: 0.69

 Peak solar angle hour: 15:00 (0.707)
 Lowest solar angle hour: 12:00 (0.647)


In [68]:
print("=" * 60)
print("SOLAR POTENTIAL STATISTICS (Raw Dataset)")
print("=" * 60)

print(f"\nSOLAR POTENTIAL:")
print(f"   - Average: {df['solar_potential'].mean():.2f}")
print(f"   - Minimum: {df['solar_potential'].min():.2f}")
print(f"   - Maximum: {df['solar_potential'].max():.2f}")
print(f"   - Std Deviation: {df['solar_potential'].std():.2f}")
print(f"   - Median: {df['solar_potential'].median():.2f} ")
print(f"   - Total: {df['solar_potential'].sum():.2f}")

# Hours with zero potential
zero_potential = len(df[df['solar_potential'] == 0])
print(f"\nZero Production Analysis:")
print(f"Hours with zero potential: {zero_potential} ({zero_potential/total*100:.2f}% of time)")

# Daily totals
daily_potential = df.groupby('date')['solar_potential'].sum()
print(f"\nDaily Solar Potential:")
print(f"   Average daily: {daily_potential.mean():.3f}")
print(f"   Best day: {daily_potential.idxmax()} ({daily_potential.max():.3f})")
print(f"   Worst day: {daily_potential.idxmin()} ({daily_potential.min():.3f})")

# Safe ratio calculation
if daily_potential.min() > 0:
    ratio = daily_potential.max() / daily_potential.min()
    print(f"   Ratio best/worst: {ratio:.1f}x")
else:
    print(f"   Ratio best/worst: Undefined (worst day has zero production)")
    
    # Alternative: compare best day to average
    if daily_potential.mean() > 0:
        print(f"   Best day vs average: {daily_potential.max()/daily_potential.mean():.1f}x average")

SOLAR POTENTIAL STATISTICS (Raw Dataset)

SOLAR POTENTIAL:
   - Average: 0.45
   - Minimum: 0.00
   - Maximum: 0.80
   - Std Deviation: 0.25
   - Median: 0.49 
   - Total: 12.61

Zero Production Analysis:
Hours with zero potential: 2 (7.14% of time)

Daily Solar Potential:
   Average daily: 0.450
   Best day: 2026-02-27 (0.800)
   Worst day: 2026-02-04 (0.000)
   Ratio best/worst: Undefined (worst day has zero production)
   Best day vs average: 1.8x average


In [72]:
# Cell 10: Worst-case analysis (for conservative estimates)
print("=" * 60)
print("WORST-CASE ANALYSIS")
print("=" * 60)

# Check if we have any non-zero solar potential
if df['solar_potential'].max() == 0:
    print("\nWARNING: All solar potential values are zero!")
    print("   This suggests there might be an issue with the data.")
    print("   Please check your CSV file.")
else:
    # Absolute worst hour
    worst_hour = df.loc[df['solar_potential'].idxmin()]
    print(f"\nAbsolute worst hour in dataset:")
    print(f"   Date: {worst_hour['date']} at {worst_hour['hour']}:00")
    print(f"   Solar potential: {worst_hour['solar_potential']:.3f}")
    print(f"   Conditions: {worst_hour['cloudcover']:.0f}% cloud cover, {worst_hour['temperature']:.1f}°C")
    print(f"   UV Index: {worst_hour['uv_index']}, Wind: {worst_hour['wind_speed']} km/h")

    # Worst day
    worst_day_idx = daily_potential.idxmin()
    worst_day_value = daily_potential.min()
    worst_day_data = df[df['date'] == worst_day_idx]
    
    print(f"\nWorst day ({worst_day_idx}):")
    print(f"   Total potential: {worst_day_value:.3f}")
    print(f"   Average cloud cover: {worst_day_data['cloudcover'].mean():.1f}%")
    print(f"   Average temperature: {worst_day_data['temperature'].mean():.1f}°C")
    print(f"   Hours with production: {len(worst_day_data[worst_day_data['solar_potential'] > 0])}")

    # Battery sizing with pre-worst-day charging simulation
    if daily_potential.mean() > 0:
        print(f"\nBATTERY STORAGE PLANNING - REALISTIC SIMULATION")
        print("=" * 60)
        
        avg_daily = daily_potential.mean()
        worst_day_position = list(daily_potential.index).index(worst_day_idx)
        
        print(f"\nWORST DAY ANALYSIS:")
        print(f"   Worst day: {worst_day_idx} with {worst_day_value:.3f} production")
        print(f"   Position in month: Day {worst_day_position + 1} of {len(daily_potential)}")
        print(f"   Average daily production: {avg_daily:.3f} kWh")
        
        # Check if worst day is the first day
        if worst_day_position == 0:
            print(f"\nWorst day is the FIRST day of the dataset - no previous days to analyze")
            print(f"   Using average daily production as reference for battery sizing")
            
            if hasattr(cfg, 'BATTERY_PARAMS'):
                min_days = cfg.BATTERY_PARAMS.get('min_days_autonomy', 1)
                battery_cost_per_kwh = cfg.BATTERY_PARAMS.get('battery_cost_per_kwh', 600)
                battery_efficiency = cfg.BATTERY_PARAMS.get('battery_efficiency', 0.9)
                
                # Use average as reference
                recommended_capacity = avg_daily * min_days * 1.5  # 50% margin
                estimated_cost = recommended_capacity * battery_cost_per_kwh
                usable_energy = recommended_capacity * battery_efficiency
                
                print(f"\nBATTERY RECOMMENDATION (based on average):")
                print(f"   Required capacity: {recommended_capacity:.2f} kWh")
                print(f"   (for {min_days} day(s) of autonomy with 50% margin)")
                print(f"   Usable energy: {usable_energy:.2f} kWh")
                print(f"   Estimated cost: €{estimated_cost:,.0f}")
        else:
            # Analyze days before worst day
            days_before = min(3, worst_day_position)  # Check up to 3 previous days
            start_idx = max(0, worst_day_position - days_before)
            pre_worst_days = daily_potential.iloc[start_idx:worst_day_position]
            
            print(f"\nENERGY FROM {len(pre_worst_days)} DAYS BEFORE WORST DAY:")
            total_before = 0
            for i, (date, value) in enumerate(pre_worst_days.items()):
                print(f"   Day -{len(pre_worst_days)-i}: {date} - {value:.3f} kWh")
                total_before += value
            
            avg_before = total_before / len(pre_worst_days) if len(pre_worst_days) > 0 else 0
            print(f"\n   Total production before worst day: {total_before:.3f} kWh")
            print(f"   Average before: {avg_before:.3f} kWh/day")
            
            # Battery simulation
            if hasattr(cfg, 'BATTERY_PARAMS'):
                battery_efficiency = cfg.BATTERY_PARAMS.get('battery_efficiency', 0.9)
                min_days = cfg.BATTERY_PARAMS.get('min_days_autonomy', 1)
                battery_cost_per_kwh = cfg.BATTERY_PARAMS.get('battery_cost_per_kwh', 600)
                
                # Calculate daily consumption (from annual consumption)
                if hasattr(cfg, 'ECON_PARAMS'):
                    daily_consumption = cfg.ECON_PARAMS.get('household_consumption', 2700) / 365
                else:
                    daily_consumption = 2700 / 365  # Default 7.4 kWh/day
                
                # Energy that could be stored from previous days (with efficiency loss)
                storage_capacity = total_before * battery_efficiency
                
                # Energy needed on worst day (consumption minus what panels produce)
                energy_needed_worst_day = max(0, daily_consumption - worst_day_value)
                
                print(f"\nBATTERY SIMULATION:")
                print(f"   Daily consumption estimate: {daily_consumption:.2f} kWh")
                print(f"   Energy needed on worst day: {energy_needed_worst_day:.2f} kWh")
                print(f"   Energy that could be stored from previous days: {storage_capacity:.2f} kWh")
                print(f"   (after {battery_efficiency*100:.0f}% efficiency)")
                
                if storage_capacity >= energy_needed_worst_day:
                    surplus = storage_capacity - energy_needed_worst_day
                    print(f"\nBATTERY WOULD HAVE SUFFICIENT CHARGE!")
                    print(f"   Surplus energy: {surplus:.2f} kWh")
                    print(f"   (could cover {surplus/daily_consumption:.1f} additional days)")
                    
                    # Recommended battery size (with 20% margin)
                    recommended_capacity = energy_needed_worst_day * 1.2
                else:
                    deficit = energy_needed_worst_day - storage_capacity
                    print(f"\nBATTERY WOULD NEED MORE CAPACITY")
                    print(f"   Energy deficit: {deficit:.2f} kWh")
                    print(f"   (would need {deficit/daily_consumption:.1f} extra days of storage)")
                    
                    # Recommended battery size (with 50% margin for safety)
                    recommended_capacity = energy_needed_worst_day * 1.5
                
                # Ensure minimum recommended capacity
                min_recommended = cfg.BATTERY_PARAMS.get('min_recommended_capacity', 2.0)
                recommended_capacity = max(recommended_capacity, min_recommended)
                
                estimated_cost = recommended_capacity * battery_cost_per_kwh
                usable_energy = recommended_capacity * battery_efficiency
                
                print(f"\nFINAL BATTERY RECOMMENDATION:")
                print(f"   Recommended capacity: {recommended_capacity:.0f} kWh")
                print(f"   Usable energy: {usable_energy:.2f} kWh")
                print(f"   Estimated cost: €{estimated_cost:,.0f}")
                print(f"   Battery lifetime: {cfg.BATTERY_PARAMS.get('battery_lifetime_years', 10)} years")
                
                # Alternative scenarios
                if hasattr(cfg, 'OUTPUT_CONFIG') and cfg.OUTPUT_CONFIG.get('verbose', False):
                    print(f"\nALTERNATIVE SCENARIOS:")
                    for days in [2, 3]:
                        alt_capacity = energy_needed_worst_day * days
                        alt_cost = alt_capacity * battery_cost_per_kwh
                        print(f"   {days} days autonomy: {alt_capacity:.2f} kWh (€{alt_cost:,.0f})")
            else:
                # Simple version if BATTERY_PARAMS doesn't exist
                print(f"\nSIMPLE BATTERY ESTIMATE:")
                print(f"   Based on {len(pre_worst_days)} days before worst day:")
                print(f"   Average production before: {avg_before:.2f} kWh/day")
                print(f"   Recommended battery: {avg_before * 2:.2f} kWh (for 2 days autonomy)")

WORST-CASE ANALYSIS

Absolute worst hour in dataset:
   Date: 2026-02-04 at 12:00
   Solar potential: 0.000
   Conditions: 100% cloud cover, 3.5°C
   UV Index: 0.0, Wind: 6 km/h

Worst day (2026-02-04):
   Total potential: 0.000
   Average cloud cover: 100.0%
   Average temperature: 3.5°C
   Hours with production: 0

BATTERY STORAGE PLANNING - REALISTIC SIMULATION

WORST DAY ANALYSIS:
   Worst day: 2026-02-04 with 0.000 production
   Position in month: Day 4 of 28
   Average daily production: 0.450 kWh

ENERGY FROM 3 DAYS BEFORE WORST DAY:
   Day -3: 2026-02-01 - 0.086 kWh
   Day -2: 2026-02-02 - 0.449 kWh
   Day -1: 2026-02-03 - 0.465 kWh

   Total production before worst day: 0.999 kWh
   Average before: 0.333 kWh/day

BATTERY SIMULATION:
   Daily consumption estimate: 7.40 kWh
   Energy needed on worst day: 7.40 kWh
   Energy that could be stored from previous days: 0.90 kWh
   (after 90% efficiency)

BATTERY WOULD NEED MORE CAPACITY
   Energy deficit: 6.50 kWh
   (would need 0.9 ex

In [70]:
# Q1 ANALYSIS FINISHED

# PREPARING FOR EXPORTING ALL THE DATA TO Q2 - SEPARATED NOTEBOOKS

# ============================================
# SAVE ALL VARIABLES FOR Q2
# ============================================

import os

print("\n" + "=" * 60)
print("SAVING ALL VARIABLES FOR Q2")
print("=" * 60)

# DEFINE OUTPUT DIRECTORY FIRST
output_dir = "notebooks_output"
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print(f"Output directory: {output_dir}")

# Get all local variables (excluding built-ins, modules, and non-pickleable objects)
import pickle
import types

all_variables = {}
for var_name in dir():
    # Skip built-in variables, modules, and private variables
    if not var_name.startswith('_') and var_name not in ['In', 'Out', 'exit', 'quit', 'get_ipython', 'pd', 'np', 'plt', 'sns', 'datetime', 'warnings', 'os', 'pickle', 'types']:
        try:
            var_value = eval(var_name)
            
            # Check if the variable is pickleable
            # Skip modules, functions, and other non-pickleable types
            if not isinstance(var_value, (types.ModuleType, types.FunctionType, types.BuiltinFunctionType, types.MethodType)):
                # Try to pickle it to be sure
                try:
                    pickle.dumps(var_value)
                    all_variables[var_name] = var_value
                except:
                    print(f"  {var_name} (cannot pickle, skipping)")
            else:
                print(f"  {var_name} (module/function, skipping)")
        except:
            print(f"  {var_name} (cannot evaluate, skipping)")

print(f"\nFound {len(all_variables)} pickle-able variables to save")

# Method 1: Save individual variables with %store
print("\nSaving individual variables with %store...")
saved_count = 0
for var_name in all_variables.keys():
    try:
        # Use %store magic command
        get_ipython().run_line_magic('store', var_name)
        saved_count += 1
        print(f"  {var_name}")
    except:
        print(f"  {var_name} (could not store)")

print(f"\nSaved {saved_count} individual variables with %store")

# Method 2: Save complete dictionary as backup
backup_file = f"{output_dir}/q1_all_variables.pkl"
with open(backup_file, 'wb') as f:
    pickle.dump(all_variables, f)

print(f"\nComplete backup saved to: {backup_file}")
file_size = os.path.getsize(backup_file) / 1024
print(f"   File size: {file_size:.1f} KB")

# Method 3: Also store the dictionary itself
%store all_variables

print("\n" + "=" * 60)
print("ALL VARIABLES SAVED SUCCESSFULLY!")
print("=" * 60)

print("\nTO ACCESS VARIABLES IN Q2:")
print("")
print("   # Method A: Load individual variables (if you know what you need)")
print("   %store -r df")
print("   %store -r daily_potential")
print("   %store -r clear_sky")
print("")
print("   # Method B: Load the complete dictionary")
print("   %store -r all_variables")
print("   df = all_variables['df']")
print("   daily_potential = all_variables['daily_potential']")
print("")
print("   # Method C: List all stored variables")
print("   %store")


SAVING ALL VARIABLES FOR Q2
Output directory: notebooks_output
  cfg (module/function, skipping)
  f (cannot pickle, skipping)
  open (module/function, skipping)
  sys (module/function, skipping)

Found 81 pickle-able variables to save

Saving individual variables with %store...
Stored 'BATTERY_PARAMS' (dict)
  BATTERY_PARAMS
Stored 'CLOUD_CATEGORIES' (dict)
  CLOUD_CATEGORIES
Stored 'ECON_PARAMS' (dict)
  ECON_PARAMS
Stored 'LOSS_PARAMS' (dict)
  LOSS_PARAMS
Stored 'OUTPUT_CONFIG' (dict)
  OUTPUT_CONFIG
Stored 'PANEL_PARAMS' (dict)
  PANEL_PARAMS
Stored 'SEASONAL_FACTORS' (dict)
  SEASONAL_FACTORS
Stored 'SELF_CONSUMPTION_PERC' (dict)
  SELF_CONSUMPTION_PERC
Stored 'SIMULATION_PARAMS' (dict)
  SIMULATION_PARAMS
Stored 'UV_CATEGORIES' (dict)
  UV_CATEGORIES
Stored 'WIND_CATEGORIES' (dict)
  WIND_CATEGORIES
Stored 'all_variables' (dict)
  all_variables
Stored 'alt_capacity' (float)
  alt_capacity
Stored 'alt_cost' (float)
  alt_cost
Stored 'avg_before' (float)
  avg_before
Stored 'avg_